In [1]:
import torch
from torch import nn, optim
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import numpy as np
from matplotlib import pyplot as plt
import statistics
from google.colab import drive
%matplotlib inline

drive.mount('/content/drive', force_remount=True)
drive_folder = '/content/drive/My Drive/'
model_name = 'cifar10_resnet50.pt'

batch_size = 128
num_epochs = 150
learning_rate = 0.1

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

Mounted at /content/drive


In [2]:
train_set = DataLoader(datasets.CIFAR10('cifar',train=True, transform=transforms.Compose(
    [
    transforms.RandomHorizontalFlip(),
    #  transforms.RandomVerticalFlip(),
    transforms.RandomResizedCrop(size=(32,32), scale = (0.8, 1.0)), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
    transforms.RandomErasing(),
    ]
), download = True), batch_size = batch_size, shuffle = True)

val_set = DataLoader(datasets.CIFAR10('cifar',train=False, transform=transforms.Compose(
    [
    #  transforms.RandomHorizontalFlip(),
    #  transforms.RandomVerticalFlip(),
    #  transforms.RandomResizedCrop(size=(32,32), scale = (0.7, 1.0)),
    #  transforms.Resize((32,32)),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
    ]
), download = True), batch_size = batch_size, shuffle = False)

print('Dataset is prepared')

Files already downloaded and verified
Files already downloaded and verified
Dataset is prepared


In [3]:
resnet = models.resnet50(pretrained=True)
model = nn.Sequential(
    *list(resnet.children())[0:-1],
    nn.Flatten(),
    nn.ReLU(),
    nn.Linear(2048,128),
    nn.ReLU(),
    nn.Linear(128,10)
)
print(device)
model = model.to(device)

model.eval()
test_x = torch.rand((1,3,32,32)).to(device)
test_y = model(test_x).to(device)
print(test_y.shape)

crit = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(model.parameters(), lr = learning_rate, momentum=0.9, weight_decay=1e-5, nesterov=True)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience = 3, verbose = True)

print('Model and utils are prepared')

cuda
torch.Size([1, 10])
Model and utils are prepared


In [4]:
total = len(val_set.dataset)
for epoch in range(num_epochs):
    model.train()
    loss_list = []

    for idx, (x, y) in enumerate(train_set):
        x, y = x.to(device), y.to(device)
        prob = model(x)
        loss = crit(prob, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())

    avg_loss = statistics.mean(loss_list)

    if epoch % 3 == 0:
        model.eval()
        total_correct = 0
        for x, y in val_set:
            x, y = x.to(device), y.to(device)
            prob = model(x)
            prid = torch.argmax(prob, dim = 1)
            total_correct += torch.sum(prid.eq(y)).float().item()
        acc = total_correct/total
        print('Epoch: {}, loss: {}, validation accuracy: {}%'.format(epoch, avg_loss, acc*100))
        scheduler.step(1-acc)
        if epoch % 10 == 0:
            torch.save(model.state_dict(),drive_folder+'epoch_{}_'.format(epoch)+model_name)
    else:
        print('Epoch: {}, loss: {}'.format(epoch, avg_loss))

Epoch: 0, loss: 1.4786740222855297, validation accuracy: 57.440000000000005%
Epoch: 1, loss: 1.2613725366494846
Epoch: 2, loss: 1.1453662654932808
Epoch: 3, loss: 1.2257003197279732, validation accuracy: 64.7%
Epoch: 4, loss: 1.04182235267766
Epoch: 5, loss: 1.0003861987682254
Epoch: 6, loss: 0.9544197810275475, validation accuracy: 71.33%
Epoch: 7, loss: 0.8757211918111347
Epoch: 8, loss: 0.9432066143931025
Epoch: 9, loss: 1.0869083541738407, validation accuracy: 57.24%
Epoch: 10, loss: 0.9321016370487945
Epoch: 11, loss: 0.8027954700657779
Epoch: 12, loss: 0.738942718566836, validation accuracy: 76.2%
Epoch: 13, loss: 0.7086094690253363
Epoch: 14, loss: 0.6699014399057764
Epoch: 15, loss: 0.6735168369987127, validation accuracy: 76.8%
Epoch: 16, loss: 0.6519143146931973
Epoch: 17, loss: 0.6651090447555112
Epoch: 18, loss: 0.6427848562407676, validation accuracy: 78.47%
Epoch: 19, loss: 0.6113026456149948
Epoch: 20, loss: 0.6535490110249775
Epoch: 21, loss: 0.6206514935969086, validat

In [0]:
# model_name = 'cifar10_resnet34.pt'
# torch.save(model.state_dict(),model_name)
# from google.colab import files
# files.download(model_name)



torch.save(model.state_dict(),drive_folder+model_name)


In [9]:
model.eval()
total_correct = 0
for x, y in val_set:
    x, y = x.to(device), y.to(device)
    prob = model(x)
    prid = torch.argmax(prob, dim = 1)
    total_correct += torch.sum(prid.eq(y)).float().item()

total = len(val_set.dataset)
acc = total_correct/total
print('total_correct: {}, total: {}, acc: {}'.format(total_correct, total, acc))

total_correct: 8669.0, total: 10000, acc: 0.8669
